<a href="https://colab.research.google.com/github/Ashoksai-tech/Physician-Note-Taker/blob/main/Physician_notemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.2 MB/s eta 0:00:00
 

# Dataset Loading and Splitting

* This section loads the dataset and splits it into training, validation, and test sets. The dataset contains medical dialogue transcripts and corresponding clinical notes

In [2]:
from datasets import load_dataset
import numpy as np

# Load the dataset
dataset = load_dataset("har1/MTS_Dialogue-Clinical_Note")

# Split dataset into train (70%), validation (15%), test (15%)
train_temp_split = dataset["train"].train_test_split(test_size=0.3, shuffle=True, seed=42)
val_test_split = train_temp_split["test"].train_test_split(test_size=0.5, shuffle=True, seed=42)

dataset["train"] = train_temp_split["train"]
dataset["validation"] = val_test_split["train"]
dataset["test"] = val_test_split["test"]

print("Updated dataset splits:", dataset.keys())
print("Train size:", len(dataset["train"]))
print("Validation size:", len(dataset["validation"]))
print("Test size:", len(dataset["test"]))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

MTS-Dialog-TrainingSet%20%28SDHP%29.csv:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

(…)Dialog-Validation%20Set%20%28SDHP%29.csv:   0%|          | 0.00/74.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1301 [00:00<?, ? examples/s]

Updated dataset splits: dict_keys(['train', 'validation', 'test'])
Train size: 910
Validation size: 195
Test size: 196


# Tokenizer and Label Configuration

* This section initializes the BioBERT tokenizer and defines the labels for Named Entity Recognition (NER). The labels include categories like SYMPTOM, DIAGNOSIS, TREATMENT, and PROGNOSIS.

In [5]:
from transformers import BertTokenizerFast

# Load BioBERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# Define labels for NER
label_list = ["O", "B-SYMPTOM", "I-SYMPTOM", "B-DIAGNOSIS", "I-DIAGNOSIS", "B-TREATMENT", "I-TREATMENT", "B-PROGNOSIS", "I-PROGNOSIS"]
label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

# Preprocessing for Named Entity Recognition (NER)

* This section preprocesses the dataset by tokenizing the dialogue and aligning entities (e.g., symptoms, diagnoses) with their corresponding tokens. It uses exact offset mapping to ensure accurate token-level labeling.

In [20]:
def preprocess_mts_for_ner(examples):
    dialogues = examples["dialogue"]
    section_texts = examples["section_text"]
    tokenized_inputs = {"input_ids": [], "attention_mask": [], "labels": []}

    for dialogue, section_text in zip(dialogues, section_texts):
        # Tokenize dialogue with offset mapping
        encodings = tokenizer(dialogue, truncation=True, padding="max_length", max_length=512, return_offsets_mapping=True)
        input_ids = encodings["input_ids"]
        attention_mask = encodings["attention_mask"]
        offsets = encodings["offset_mapping"]
        labels = [label2id["O"]] * len(input_ids)
        dialogue_lower = dialogue.lower()

        # Parse section_text for entities
        section_lines = section_text.lower().split("\n")
        entities = []

        # Symptoms
        for line in section_lines:
            if "symptoms:" in line:
                symptoms = [s.strip() for s in line.split("symptoms:")[1].strip().split(",")]
                for symptom in symptoms:
                    if symptom and symptom in dialogue_lower:
                        start = dialogue_lower.index(symptom)
                        end = start + len(symptom)
                        entities.append((start, end, "SYMPTOM"))


        for line in section_lines:
            if "diagnosis:" in line:
                diagnosis_text = line.split("diagnosis:")[1].strip()
                diagnoses = [d.strip() for d in diagnosis_text.split(",")]
                for diagnosis in diagnoses:
                    if diagnosis in dialogue_lower:
                        start = dialogue_lower.index(diagnosis)
                        end = start + len(diagnosis)
                        entities.append((start, end, "DIAGNOSIS"))
                    else:

                        keywords = diagnosis.split()
                        for keyword in keywords:
                            if keyword in dialogue_lower:
                                start = dialogue_lower.index(keyword)
                                end = start + len(keyword)
                                entities.append((start, end, "DIAGNOSIS"))


        treatment_keywords = ["prescribed", "sessions", "plan of action:"]
        for line in section_lines:
            if any(kw in line for kw in treatment_keywords):
                if "plan of action:" in line and "n/a" not in line:
                    treatments = [t.strip() for t in line.split("plan of action:")[1].strip().split(",")]
                else:
                    treatments = [line.strip()]  # Treat the whole line as a potential treatment if keywords match
                for treatment in treatments:
                    if treatment in dialogue_lower:
                        start = dialogue_lower.index(treatment)
                        end = start + len(treatment)
                        entities.append((start, end, "TREATMENT"))
                    else:
                        # Check for partial matches (e.g., "physiotherapy" from "ten sessions of physiotherapy")
                        keywords = treatment.split()
                        for keyword in keywords:
                            if keyword in dialogue_lower:
                                start = dialogue_lower.index(keyword)
                                end = start + len(keyword)
                                entities.append((start, end, "TREATMENT"))

        # Prognosis
        prognosis_terms = ["recover", "improving", "better", "heal", "resolve"]
        for term in prognosis_terms:
            if term in dialogue_lower:
                start = dialogue_lower.index(term)
                end = start + len(term)
                entities.append((start, end, "PROGNOSIS"))

        # Align entities to tokens
        entities.sort(key=lambda x: x[0])  # Sort by start position
        for start, end, entity_type in entities:
            for i, (offset_start, offset_end) in enumerate(offsets):
                if offset_start >= start and offset_end <= end and offset_end > 0:
                    if labels[i] == label2id["O"]:
                        labels[i] = label2id[f"B-{entity_type}"]
                    # Extend to subsequent tokens if entity spans multiple
                    j = i + 1
                    while j < len(offsets) and offsets[j][0] < end and offsets[j][1] > 0:
                        if labels[j] == label2id["O"]:
                            labels[j] = label2id[f"I-{entity_type}"]
                        j += 1

        tokenized_inputs["input_ids"].append(input_ids)
        tokenized_inputs["attention_mask"].append(attention_mask)
        tokenized_inputs["labels"].append(labels)

    return tokenized_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(
    preprocess_mts_for_ner,
    batched=True,
    remove_columns=dataset["train"].column_names
)

tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

# Tuning the BioBERT NER Model

* This section initializes the BioBERT model for token classification and trains it on the preprocessed dataset. It uses a custom compute_metrics function to evaluate the model's performance during training. The model is fine-tuned to recognize medical entities like symptoms, diagnoses, and treatments.

In [24]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score

# Load BioBERT model
model = AutoModelForTokenClassification.from_pretrained(
    "dmis-lab/biobert-base-cased-v1.1",
    num_labels=len(label_list),
    label2id=label2id,
    id2label=id2label
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./biobert_ner_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,  # Lowered for better convergence
    per_device_train_batch_size=8,  # Increased for faster training
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Reduced to prevent overfitting
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    pred_labels = [[id2label[p] for p, l in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]
    return {
        "precision": precision_score(true_labels, pred_labels),
        "recall": recall_score(true_labels, pred_labels),
        "f1": f1_score(true_labels, pred_labels),
        "report": classification_report(true_labels, pred_labels)
    }

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train and evaluate
trainer.train()
test_results = trainer.evaluate(tokenized_dataset["test"])
print("Test Results:", test_results)

# Save model
trainer.save_model("./biobert_ner_finetuned")
tokenizer.save_pretrained("./biobert_ner_finetuned")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Report
1,0.050500,0.039081,0.000000,0.000000,0.000000,precision recall f1-score support DIAGNOSIS 0.00 0.00 0.00 120 PROGNOSIS 0.00 0.00 0.00 9 SYMPTOM 0.00 0.00 0.00 53 TREATMENT 0.00 0.00 0.00 325 micro avg 0.00 0.00 0.00 507 macro avg 0.00 0.00 0.00 507 weighted avg 0.00 0.00 0.00 507
2,0.043900,0.033194,0.607595,0.094675,0.163823,precision recall f1-score support DIAGNOSIS 0.06 0.01 0.01 120 PROGNOSIS 0.00 0.00 0.00 9 SYMPTOM 0.00 0.00 0.00 53 TREATMENT 0.75 0.14 0.24 325 micro avg 0.61 0.09 0.16 507 macro avg 0.20 0.04 0.06 507 weighted avg 0.49 0.09 0.16 507
3,0.034000,0.029912,0.379310,0.130178,0.193833,precision recall f1-score support DIAGNOSIS 0.05 0.03 0.03 120 PROGNOSIS 0.00 0.00 0.00 9 SYMPTOM 0.00 0.00 0.00 53 TREATMENT 0.54 0.19 0.29 325 micro avg 0.38 0.13 0.19 507 macro avg 0.15 0.05 0.08 507 weighted avg 0.36 0.13 0.19 507
4,0.024100,0.030713,0.388715,0.244576,0.300242,precision recall f1-score support DIAGNOSIS 0.12 0.07 0.09 120 PROGNOSIS 0.62 0.89 0.73 9 SYMPTOM 0.26 0.26 0.26 53 TREATMENT 0.52 0.29 0.37 325 micro avg 0.39 0.24 0.30 507 macro avg 0.38 0.38 0.36 507 weighted avg 0.40 0.24 0.30 507
5,0.019200,0.035141,0.285223,0.327416,0.304867,precision recall f1-score support DIAGNOSIS 0.13 0.08 0.10 120 PROGNOSIS 0.62 0.89 0.73 9 SYMPTOM 0.26 0.43 0.32 53 TREATMENT 0.31 0.38 0.34 325 micro avg 0.29 0.33 0.30 507 macro avg 0.33 0.45 0.37 507 weighted avg 0.27 0.33 0.29 507
6,0.015300,0.033389,0.374026,0.284024,0.322870,precision recall f1-score support DIAGNOSIS 0.12 0.07 0.09 120 PROGNOSIS 0.62 0.89 0.73 9 SYMPTOM 0.40 0.32 0.36 53 TREATMENT 0.44 0.34 0.38 325 micro avg 0.37 0.28 0.32 507 macro avg 0.39 0.41 0.39 507 weighted avg 0.36 0.28 0.32 507
7,0.012100,0.038560,0.292484,0.353057,0.319929,precision recall f1-score support DIAGNOSIS 0.12 0.13 0.13 120 PROGNOSIS 0.62 0.89 0.73 9 SYMPTOM 0.25 0.40 0.31 53 TREATMENT 0.35 0.41 0.38 325 micro avg 0.29 0.35 0.32 507 macro avg 0.33 0.46 0.38 507 weighted avg 0.29 0.35 0.32 507
8,0.008500,0.040651,0.301855,0.353057,0.325455,precision recall f1-score support DIAGNOSIS 0.14 0.12 0.13 120 PROGNOSIS 0.73 0.89 0.80 9 SYMPTOM 0.25 0.47 0.32 53 TREATMENT 0.35 0.41 0.37 325 micro avg 0.30 0.35 0.33 507 macro avg 0.37 0.47 0.41 507 weighted avg 0.29 0.35 0.32 507
9,0.007900,0.039508,0.344595,0.301775,0.321767,precision recall f1-score support DIAGNOSIS 0.14 0.11 0.12 120 PROGNOSIS 0.80 0.89 0.84 9 SYMPTOM 0.31 0.40 0.35 53 TREATMENT 0.40 0.34 0.37 325 micro avg 0.34 0.30 0.32 507 macro avg 0.41 0.43 0.42 507 weighted avg 0.34 0.30 0.32 507
10,0.008200,0.040610,0.334043,0.309665,0.321392,precision recall f1-score support DIAGNOSIS 0.14 0.11 0.12 120 PROGNOSIS 0.73 0.89 0.80 9 SYMPTOM 0.29 0.42 0.34 53 TREATMENT 0.39 0.35 0.37 325 micro avg 0.33 0.31 0.32 507 macro avg 0.39 0.44 0.41 507 weighted avg 0.33 0.31 0.32 507


Test Results: {'eval_loss': 0.03946970775723457, 'eval_precision': 0.3228228228228228, 'eval_recall': 0.39019963702359345, 'eval_f1': 0.35332785538208705, 'eval_report': '              precision    recall  f1-score   support\n\n   DIAGNOSIS       0.20      0.20      0.20       142\n   PROGNOSIS       0.64      0.82      0.72        17\n     SYMPTOM       0.18      0.40      0.25        53\n   TREATMENT       0.39      0.45      0.42       339\n\n   micro avg       0.32      0.39      0.35       551\n   macro avg       0.35      0.47      0.40       551\nweighted avg       0.33      0.39      0.35       551\n', 'eval_runtime': 6.3616, 'eval_samples_per_second': 30.81, 'eval_steps_per_second': 3.93, 'epoch': 10.0}


('./biobert_ner_finetuned/tokenizer_config.json',
 './biobert_ner_finetuned/special_tokens_map.json',
 './biobert_ner_finetuned/vocab.txt',
 './biobert_ner_finetuned/added_tokens.json',
 './biobert_ner_finetuned/tokenizer.json')

# Processing and Pipeline Setup

* This section sets up the processing pipeline for NER, sentiment analysis, and intent detection. It uses the fine-tuned BioBERT model for NER and a medical-specific sentiment model for sentiment analysis

In [25]:
from transformers import pipeline, BertForTokenClassification, BertTokenizerFast
from sklearn.feature_extraction.text import TfidfVectorizer

# Load fine-tuned NER model
ner_model = BertForTokenClassification.from_pretrained("./biobert_ner_finetuned")
ner_tokenizer = BertTokenizerFast.from_pretrained("./biobert_ner_finetuned")
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple")

# Other pipelines
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_analyzer = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")# Medical-specific model
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
intents = ["Seeking reassurance", "Reporting symptoms", "Expressing concern"]

Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


# Command-Line Interface (CLI) for Transcript Processing

* This section provides a CLI for users to input medical transcripts and receive structured outputs, including medical details, sentiment analysis, and SOAP notes. It uses the processing pipeline to generate results in real-time.

In [26]:
def process_transcript(transcript):
    # NER
    ner_results = ner_pipeline(transcript)
    entity_dict = {"Symptoms": [], "Diagnosis": [], "Treatments": [], "Prognosis": []}
    for entity in ner_results:
        label = entity["entity_group"]
        text = entity["word"]
        if "SYMPTOM" in label:
            entity_dict["Symptoms"].append(text)
        elif "DIAGNOSIS" in label:
            entity_dict["Diagnosis"].append(text)
        elif "TREATMENT" in label:
            entity_dict["Treatments"].append(text)
        elif "PROGNOSIS" in label:
            entity_dict["Prognosis"].append(text)

    # Summary and keywords
    cleaned = " ".join(transcript.split("\n"))
    summary = summarizer(cleaned, max_length=50, min_length=20)[0]["summary_text"]
    vectorizer = TfidfVectorizer(stop_words="english", max_features=5)
    tfidf_matrix = vectorizer.fit_transform([cleaned])
    keywords = vectorizer.get_feature_names_out().tolist()

    medical_details = {
        "Patient_Name": "Unknown" if "Mr." not in transcript and "Ms." not in transcript else transcript.split("Mr." if "Mr." in transcript else "Ms.")[1].split()[0],
        "Symptoms": list(set(entity_dict["Symptoms"])),
        "Diagnosis": list(set(entity_dict["Diagnosis"])),
        "Treatments": list(set(entity_dict["Treatments"])),
        "Current_Status": "Occasional symptoms" if "occasional" in cleaned.lower() else "Improving",
        "Prognosis": list(set(entity_dict["Prognosis"])) or ["Full recovery expected"]
    }

    # Sentiment and Intent with emotion-based mapping
    patient_lines = [line.split("Patient:")[1].strip() for line in transcript.split("\n") if "Patient:" in line]
    sentiment_intent = []
    for line in patient_lines:
        sentiment_result = sentiment_analyzer(line)[0]
        emotion = sentiment_result["label"]
        score = sentiment_result["score"]

        # Map emotions to your sentiment classes
        if emotion in ["fear", "sadness"] and score > 0.7:
            sentiment = "Anxious"
        elif emotion in ["joy", "love"] and score > 0.7:
            sentiment = "Reassured"
        else:
            sentiment = "Neutral"

        # Intent detection with zero-shot classification
        intent_result = zero_shot(line, intents)
        intent = intent_result["labels"][0]
        sentiment_intent.append({"Line": f"Patient: {line}", "Sentiment": sentiment, "Intent": intent})

    # SOAP Note
    soap = {
        "Subjective": {
            "Chief_Complaint": " and ".join(medical_details["Symptoms"]) or "Not specified",
            "History_of_Present_Illness": summary
        },
        "Objective": {
            "Physical_Exam": next((line.split("Physician:")[1].strip() for line in transcript.split("\n") if "examination" in line.lower() or "check" in line.lower() or "vitals" in line.lower()), "Not recorded"),
            "Observations": "Patient appears stable."
        },
        "Assessment": {
            "Diagnosis": medical_details["Diagnosis"] or ["Pending evaluation"],
            "Severity": "Mild, improving" if "better" in cleaned.lower() else "Under evaluation"
        },
        "Plan": {
            "Treatment": medical_details["Treatments"] or ["Monitor and manage symptoms"],
            "Follow-Up": "Return if symptoms worsen or persist beyond six months."
        }
    }

    return medical_details, sentiment_intent, soap

In [ ]:
def run_cli():
    print("Physician Notetaker CLI with Fine-Tuned BioBERT NER")

    # Smaller sample transcript
    sample_transcript = """Physician: Good morning, Ms. Smith. How are you feeling today?
Patient: Morning, doctor. I’ve had a sore throat and fever for two days.
Physician: Any treatment so far?
Patient: I’ve been taking ibuprofen, but it’s not helping much.
Physician: Let’s check your throat. [Pause] It looks like a mild infection.
Physician: Rest and fluids should help you recover in a few days."""

    print("\nProcessing sample transcript:")
    print(sample_transcript)

    # Process the transcript with error handling
    try:
        med_details, sent_intent, soap = process_transcript(sample_transcript)
        print("\nMedical Details:")
        print(med_details)
        print("\nSentiment & Intent:")
        for item in sent_intent:
            print(item)
        print("\nSOAP Note:")
        print(soap)
    except Exception as e:
        print(f"Error processing sample transcript: {e}")

    # Interactive CLI loop with error handling
    while True:
        print("\nEnter your own transcript (or 'exit' to quit):")
        user_input = input("Transcript: ")
        if user_input.lower() == "exit":
            break
        if not user_input.strip():
            print("Please enter a valid transcript.")
            continue

        try:
            med_details, sent_intent, soap = process_transcript(user_input)
            print("\nMedical Details:")
            print(med_details)
            print("\nSentiment & Intent:")
            for item in sent_intent:
                print(item)
            print("\nSOAP Note:")
            print(soap)
        except Exception as e:
            print(f"Error processing user transcript: {e}")

if __name__ == "__main__":
    run_cli()

Physician Notetaker CLI with Fine-Tuned BioBERT NER

Processing sample transcript:
Physician: Good morning, Ms. Smith. How are you feeling today?
Patient: Morning, doctor. I’ve had a sore throat and fever for two days.
Physician: Any treatment so far?
Patient: I’ve been taking ibuprofen, but it’s not helping much.
Physician: Let’s check your throat. [Pause] It looks like a mild infection.
Physician: Rest and fluids should help you recover in a few days.

Medical Details:
{'Patient_Name': 'Smith.', 'Symptoms': ['sore throat', 'fever'], 'Diagnosis': ['infection'], 'Treatments': ['in', 'fluids', 'ibuprofen', 'patient :', 'rest', 'for', 'recover'], 'Current_Status': 'Improving', 'Prognosis': ['Full recovery expected']}

Sentiment & Intent:
{'Line': 'Patient: Morning, doctor. I’ve had a sore throat and fever for two days.', 'Sentiment': 'Anxious', 'Intent': 'Reporting symptoms'}
{'Line': 'Patient: I’ve been taking ibuprofen, but it’s not helping much.', 'Sentiment': 'Reassured', 'Intent': '